In [ ]:
import os
import sys

import scipy as sp
from matplotlib import pyplot as plt
from matplotlib.patches import Rectangle
import pandas as pd
import numpy as np

from jax import vmap
import jax.numpy as jnp

CWD = os.path.abspath("")
sys.path.append(CWD)
sys.path.append(os.path.join(CWD, "../.."))
from plt_settings import plt_settings

full_width = 5.5
ratio = 1 / 1.618

In [ ]:
def sample_returns(mu_samples, sigma_samples):
    def psd(X):
        D, V = np.linalg.eig(X)
        return V @ np.diag(np.clip(D, 0, None)) @ V.T

    returns = [
        sp.stats.multivariate_normal.rvs(mean=mu, cov=psd(sigma))
        for mu, sigma in zip(mu_samples, sigma_samples)
    ]
    return pd.DataFrame(returns, columns=stocks.columns.values)


# Read stock time-series and create the (log-)returns matrix
stock_files = {}
stocks_label = ["TSLA", "NVDA", "JNJ", "AAPL", "GOOG", "BRK-B", "LLY"]
for f in [os.path.join(CWD, f"../data/{stock}.csv") for stock in stocks_label]:
    stock_files[os.path.basename(f)[: -len(".csv")]] = pd.read_csv(f, index_col="Date")["Adj Close"]
stocks = pd.DataFrame(stock_files)
stocks = 100 * np.log(stocks).diff()
stocks.dropna(inplace=True)
n_stocks = stocks.shape[1]

# Frequentist estimates of the mean and covariance for returns
mu_hat = stocks.mean()
sigma_hat = stocks.cov()

print("")
print(f"Average (log-)return:\n{mu_hat}")

In [ ]:
ITERATIONS = int(4e5)
BURN_IN = ITERATIONS // 2

idx_stocks = ["NVDA", "LLY"]
mask = jnp.array([jnp.eye(n_stocks)[stocks.columns == stock, :].squeeze() for stock in idx_stocks])
desired_vals = jnp.array(
    [mu_hat[stock] * (1.2 if mu_hat[stock] >= 0 else 0.8) for stock in idx_stocks]
)


def eqconst(mu):
    return jnp.array([jnp.inner(mu, mask) - desired_vals]).squeeze()


sampling_data = np.load(os.path.join(CWD, "nvda_lly.npz"))
pdlmc_mu_samples = sampling_data["pdlmc_mu"][BURN_IN:]
pdlmc_sigma_samples = sampling_data["pdlmc_sigma"][BURN_IN:]
pdlmc_nu = sampling_data["pdlmc_nu"]
lmc_mu_samples = sampling_data["lmc_mu"][BURN_IN:]
lmc_sigma_samples = sampling_data["lmc_sigma"][BURN_IN:]

eq_slacks = vmap(eqconst)(sampling_data["pdlmc_mu"])

In [ ]:
print(f"Average return per stock:")
print(
    pd.DataFrame(
        {
            "Frequentist": mu_hat.to_numpy(),
            "LMC": lmc_mu_samples.mean(axis=0),
            "PD-LMC": pdlmc_mu_samples.mean(axis=0),
        },
        index=stocks_label,
    )
)
print(f"Standard deviation of the Bayesian estimates:")
print(
    pd.DataFrame(
        {
            "LMC": lmc_mu_samples.std(axis=0),
            "PD-LMC": pdlmc_mu_samples.std(axis=0),
        },
        index=stocks_label,
    )
)

print("")
print(f"Average volatility per stock:")
print(
    pd.DataFrame(
        {
            "Frequentist": np.diag(sigma_hat.to_numpy()),
            "LMC": np.diag(lmc_sigma_samples.mean(axis=0)),
            "PD-LMC": np.diag(pdlmc_sigma_samples.mean(axis=0)),
        },
        index=stocks_label,
    )
)
print(f"Standard deviation of the Bayesian estimates:")
print(
    pd.DataFrame(
        {
            "LMC": np.diag(lmc_sigma_samples.std(axis=0)),
            "PD-LMC": np.diag(pdlmc_sigma_samples.std(axis=0)),
        },
        index=stocks_label,
    )
)

print(f"Relative return increase:")
print(
    pd.DataFrame(
        {
            "PD-LMC/LMC": pdlmc_mu_samples.mean(axis=0) / lmc_mu_samples.mean(axis=0),
        },
        index=stocks_label,
    )
)

In [ ]:
plt_settings["figure.figsize"] = (full_width / 2, ratio * full_width / 2)

with plt.rc_context(plt_settings):
    _, axs = plt.subplots(1, 1, dpi=300)
    axs.plot(pdlmc_nu, label=idx_stocks)
    axs.grid()
    axs.set_xlabel("Iterations")
    axs.set_ylabel(r"Dual variables ($\nu$)")
    axs.ticklabel_format(scilimits=(-3, 2))
    plt.legend()

    plt.show()

In [ ]:
plt_settings["figure.figsize"] = (full_width / 2, ratio * full_width / 2)

eq_cum_mean = np.cumsum(eq_slacks, axis=0) / np.expand_dims(
    np.arange(1, eq_slacks.shape[0] + 1), axis=1
)

with plt.rc_context(plt_settings):
    _, axs = plt.subplots(1, 1, dpi=300)
    axs.plot(eq_cum_mean, label=idx_stocks)
    axs.grid()
    axs.set_xlabel("Iterations")
    axs.set_ylabel("Ergodic constraint slacks")
    axs.ticklabel_format(scilimits=(-2, 2))
    plt.legend()

    plt.show()

In [ ]:
plt_settings["figure.figsize"] = (full_width, ratio * full_width / 2)

linewidths = 0.6

lmc_var_samples = np.array(list(map(np.diag, lmc_sigma_samples)))
pdlmc_var_samples = np.array(list(map(np.diag, pdlmc_sigma_samples)))

with plt.rc_context(plt_settings):
    fig, axs = plt.subplots(1, 2, dpi=300)

    axs[0].set_ylabel(r"(Log)return")
    parts = axs[0].violinplot(dataset=lmc_mu_samples, showmeans=True, side="low")
    for key in ["cmins", "cmaxes", "cbars", "cmeans"]:
        parts[key].set(lw=linewidths)

    parts = axs[0].violinplot(dataset=pdlmc_mu_samples, showmeans=True, side="high")
    for key in ["cmins", "cmaxes", "cbars", "cmeans"]:
        parts[key].set(lw=linewidths)

    axs[1].set_ylabel(r"Volatility")
    parts = axs[1].violinplot(dataset=lmc_var_samples, showmeans=True, side="low")
    for key in ["cmins", "cmaxes", "cbars", "cmeans"]:
        parts[key].set(lw=linewidths)

    parts = axs[1].violinplot(dataset=pdlmc_var_samples, showmeans=True, side="high")
    for key in ["cmins", "cmaxes", "cbars", "cmeans"]:
        parts[key].set(lw=linewidths)

    for ax in axs:
        ax.grid()
        ax.set_xticks(np.arange(1, len(stocks_label) + 1), labels=stocks_label)

    # inset Axes....
    axins = axs[1].inset_axes(
        [0.43, 0.45, 0.51, 0.51],
        xlim=(0, 6),
        ylim=(1, 4.9),
    )
    parts = axins.violinplot(dataset=lmc_var_samples[:, 2:], showmeans=True, side="low")
    for key in ["cmins", "cmaxes", "cbars", "cmeans"]:
        parts[key].set(lw=linewidths)
    parts = axins.violinplot(dataset=pdlmc_var_samples[:, 2:], showmeans=True, side="high")
    for key in ["cmins", "cmaxes", "cbars", "cmeans"]:
        parts[key].set(lw=linewidths)

    axins.set_facecolor("w")
    axins.grid()
    axins.set_xticks(
        np.arange(1, 6),
        labels=stocks_label[2:],
        fontsize=6,
        rotation_mode="anchor",
        rotation=35,
        ha="right",
    )

    axs[0].legend(
        [Rectangle((0, 0), 1, 1, color=f"C{n}", alpha=1.0) for n in range(2)],
        ["LMC", "PD-LMC"],
        handlelength=0.7,
        loc="upper right",
        ncol=2,
    )

    plt.show()

In [ ]:
plt_settings["figure.figsize"] = (full_width, ratio * full_width / 2)

linewidths = 0.6

sampling_data_all = np.load(os.path.join(CWD, "all_stocks.npz"))
pdlmc_mu_samples_all = sampling_data_all["pdlmc_mu"][BURN_IN:]
pdlmc_sigma_samples_all = sampling_data_all["pdlmc_sigma"][BURN_IN:]
pdlmc_var_samples_all = np.array(list(map(np.diag, pdlmc_sigma_samples_all)))

with plt.rc_context(plt_settings):
    fig, axs = plt.subplots(1, 2, dpi=300)

    axs[0].set_ylabel(r"(Log)return")
    parts = axs[0].violinplot(dataset=pdlmc_mu_samples_all, showmeans=True, side="low")
    for key in ["cmins", "cmaxes", "cbars", "cmeans"]:
        parts[key].set(lw=linewidths)

    parts = axs[0].violinplot(dataset=pdlmc_mu_samples, showmeans=True, side="high")
    for key in ["cmins", "cmaxes", "cbars", "cmeans"]:
        parts[key].set(lw=linewidths)

    axs[1].set_ylabel(r"Volatility")
    parts = axs[1].violinplot(dataset=pdlmc_var_samples_all, showmeans=True, side="low")
    for key in ["cmins", "cmaxes", "cbars", "cmeans"]:
        parts[key].set(lw=linewidths)

    parts = axs[1].violinplot(dataset=pdlmc_var_samples, showmeans=True, side="high")
    for key in ["cmins", "cmaxes", "cbars", "cmeans"]:
        parts[key].set(lw=linewidths)

    for ax in axs:
        ax.grid()
        ax.set_xticks(np.arange(1, len(stocks_label) + 1), labels=stocks_label)

    # inset Axes....
    axins = axs[1].inset_axes(
        [0.43, 0.45, 0.51, 0.51],
        xlim=(0, 6),
        ylim=(1, 5.2),
    )
    parts = axins.violinplot(dataset=pdlmc_var_samples_all[:, 2:], showmeans=True, side="low")
    for key in ["cmins", "cmaxes", "cbars", "cmeans"]:
        parts[key].set(lw=linewidths)
    parts = axins.violinplot(dataset=pdlmc_var_samples[:, 2:], showmeans=True, side="high")
    for key in ["cmins", "cmaxes", "cbars", "cmeans"]:
        parts[key].set(lw=linewidths)

    axins.set_facecolor("w")
    axins.grid()
    axins.set_xticks(
        np.arange(1, 6),
        labels=stocks_label[2:],
        fontsize=6,
        rotation_mode="anchor",
        rotation=35,
        ha="right",
    )

    axs[0].legend(
        [Rectangle((0, 0), 1, 1, color=f"C{n}", alpha=1.0) for n in range(2)],
        ["All stocks", "NVDA, LLY"],
        handlelength=0.7,
        loc="upper right",
        ncol=2,
    )

    plt.show()